In [87]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd
from tqdm import tqdm
import numpy as np

from pangres import upsert
from sqlalchemy import text, create_engine

import json

load_dotenv()

True

In [3]:
db_username=os.environ.get("db_username")
db_password=os.environ.get("db_password")
db_host=os.environ.get("db_host")
db_port=os.environ.get("db_port")
# db_name=os.environ.get("db_name")
db_name = 'lol_analytics'


def create_db_connection_string(db_username: str, db_password: str, db_host: str, db_port: int, db_name: str):
    connection_url = f"postgresql+psycopg2://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}"
    return connection_url

In [2]:
def get_match_timeline(match_id, api_key):
    link = f'https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}/timeline?api_key={api_key}'
    response = requests.get(link)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
    return None

In [31]:
def retrieve_match_data(match_id, api_key):
    link = f'https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}'
    response = requests.get(link)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
    return response.json()

In [8]:
match_id = 'NA1_5402091165'

api_key = os.getenv("riot_api_key")
with open('match_timeline_metadata.json', 'w') as f:
    json.dump(get_match_timeline(match_id, api_key), f)

## Exploring data

In [ ]:
retrieve_match_data(match_id, api_key)['info']['gameDuration']/60
# the game is 22.8 minutes long and there are 24 frames captured

22.8

In [10]:
match_timeline = get_match_timeline(match_id=match_id, api_key=api_key)

In [ ]:
participants = match_timeline['info']['participants']

In [117]:
simplified_participants = {}
for participant in participants:
    # print(participant)
    simplified_participants[participant['participantId']] = participant['puuid']

# simplified_participants

In [147]:
match_timeline['info']['frames'][0]['timestamp']# match_timeline

0

In [ ]:
def process_timeline_data(match_timeline):
    
    participants_timeline = []
    match_metadata = match_timeline['metadata']
    frames = match_timeline['info']['frames']

    for frame in frames:
        participant_timeline = {}
        participant_timeline['data_version'] = match_metadata['dataVersion']
        participant_timeline['match_id'] = match_metadata['matchId']
        participant_timeline['frame_interval'] = match_metadata['frameInterval']/1000
        participant_timeline['end_of_game_result'] = frame['endOfGameResult']
        # convert into minutes
        participant_timeline['timestamp'] = round(frame['timestamp']/1000/60, 2)
        for participant_id, participant_dto in frame['participantFrames'].items():
            participant_timeline['participant_id'] = int(participant_id)
            participant_timeline['puuid'] = simplified_participants[int(participant_id)]
            participant_timeline['current_gold'] = participant_dto['currentGold']
            participant_timeline['level'] = participant_dto['level']
            participant_timeline['jungle_minions_killed'] = participant_dto['jungleMinionsKilled']
            participant_timeline['minions_killed'] = participant_dto['minionsKilled']
            participant_timeline['total_gold'] = participant_dto['totalGold']
            participant_timeline['xp'] = participant_dto['xp']

            champion_stats = participant_dto['championStats']
            participant_timeline['health'] = champion_stats['health']
            participant_timeline['health_max'] = champion_stats['healthMax']
            participant_timeline['mana'] = champion_stats['power']
            participant_timeline['mana_max'] = champion_stats['powerMax']
            

            player_position = participant_dto['position']
            participant_timeline['x'] = player_position['x']
            participant_timeline['y'] = player_position['y']

        participants_timeline.append(participant_timeline)

    return participants_timeline

In [148]:
def retrieve_match_data(match_id, api_key):
    link = f'https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}'
    response = requests.get(link)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
    return response.json()

In [133]:
pd.DataFrame(process_timeline_data(match_timeline))

,match_id,timestamp,participant_id,puuid,current_gold,level,jungle_minions_killed,minions_killed,total_gold,xp,health,health_max,mana,mana_max,x,y
0,NA1_5402091165,0.00,10,yAz2yNwmR98ICucUS1GIg2IaUHPmsH5liWAaVlgEulgvIU...,100,1,0,0,500,0,610,610,480,480,14281,14199
1,NA1_5402091165,1.00,10,yAz2yNwmR98ICucUS1GIg2IaUHPmsH5liWAaVlgEulgvIU...,0,1,0,0,500,0,675,675,436,480,11806,5058
2,NA1_5402091165,2.00,10,yAz2yNwmR98ICucUS1GIg2IaUHPmsH5liWAaVlgEulgvIU...,62,1,0,1,562,220,675,675,434,505,12115,1882
3,NA1_5402091165,3.00,10,yAz2yNwmR98ICucUS1GIg2IaUHPmsH5liWAaVlgEulgvIU...,263,2,0,1,763,541,746,746,303,571,11203,3269
4,NA1_5402091165,4.00,10,yAz2yNwmR98ICucUS1GIg2IaUHPmsH5liWAaVlgEulgvIU...,526,3,0,1,1026,964,293,821,132,639,12930,1912
5,NA1_5402091165,5.00,10,yAz2yNwmR98ICucUS1GIg2IaUHPmsH5liWAaVlgEulgvIU...,341,3,0,1,1241,1034,821,821,516,689,12130,4770
6,NA1_5402091165,6.00,10,yAz2yNwmR98ICucUS1GIg2IaUHPmsH5liWAaVlgEulgvIU...,639,4,0,2,1539,1228,768,899,410,733,11393,1447
7,NA1_5402091165,7.00,10,yAz2yNwmR98ICucUS1GIg2IaUHPmsH5liWAaVlgEulgvIU...,14,4,0,3,1714,1651,899,899,1083,1083,13989,12051
8,NA1_5402091165,8.00,10,yAz2yNwmR98ICucUS1GIg2IaUHPmsH5liWAaVlgEulgvIU...,205,4,0,3,2205,1697,969,969,1083,1083,12918,11514
9,NA1_5402091165,9.00,10,yAz2yNwmR98ICucUS1GIg2IaUHPmsH5liWAaVlgEulgvIU...,521,5,0,11,2521,2151,1029,1050,1102,1102,11491,3831
